In [0]:
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-cdp-bu-finance-dev`.curated.EPMParentChildHierarchy
  WHERE
    `IsBaseRecord` = 1
    AND adjacencypath like 'Entity|Shared_Mbr_Entity_Structure|Management|BU_NA|%'
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
extract as (
  SELECT
    'Indirect HC' as jobtype,
    CASE
      WHEN Breakdown2Id = 'TKE101' THEN 'NI'
      WHEN Breakdown2Id = 'TKE102' THEN 'MOD'
      WHEN Breakdown2Id = 'TKE1031' THEN 'SERV'
      WHEN Breakdown2Id = 'TKE1032' THEN 'REPAIR'
    END AS LOB,
    branch.alias as entity,
    TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
    fte
  FROM
    `na-cdp-bu-hr-dev`.curated.fte as fte
      JOIN onlybaseentities as child
        on child.child = fte.epmbranch
      LEFT JOIN branchnames as branch
        on branch.backendname = fte.epmbranch
  WHERE
    CostType IN ('COS INDIRECT','G&A','SELLING')
    and Breakdown2Id IN ('TKE101', 'TKE102', 'TKE1031', 'TKE1032')
    and fte <> 0
),
indirect_entity as (
  SELECT
    jobtype as Account,
    LOB,
    `Calendar Date`,
    entity as Entity,
    sum(fte) as Value
  FROM
    extract
  GROUP BY
    ALL
),
indirect_buna as (
  SELECT
    jobtype as Account,
    LOB,
    `Calendar Date`,
    'BU North America' as Entity,
    sum(fte) as Value
  FROM
    extract
  GROUP BY
    ALL
)
SELECT
  *,
  'Actuals' as Scenario,
  'Indirect HC' as Source
FROM
  (
    SELECT
      *
    from
      indirect_entity
    UNION ALL
    SELECT
      *
    from
      indirect_buna
  )